# Homology Modelling of Side Chains

In [1]:
from pathlib import Path

# change accordingly
GENERATED_CLS = GENERATED_CLS if "GENERATED_CLS" in locals() else "BLDB"
GENERATED_SID = GENERATED_SID if "GENERATED_SID" in locals() else "FM_0"
assert GENERATED_CLS is not None
assert GENERATED_SID is not None

MIN_HOMOLOGY = MIN_HOMOLOGY if "MIN_HOMOLOGY" in locals() else 0.35
MIN_TM_SCORE = MIN_TM_SCORE if "MIN_TM_SCORE" in locals() else 0.8
MAX_TEMPLATES = MAX_TEMPLATES if "MAX_TEMPLATES" in locals() else 5
print(
    f"Homology Modelling: {GENERATED_CLS}/{GENERATED_SID}. Min Homology = {MIN_HOMOLOGY}. Maximum Number of Templates to Use = {MAX_TEMPLATES}.")

GENERATED_PDB = Path(f"../../data/{GENERATED_CLS}/generated/BQ/{GENERATED_SID}.pdb")
GENERATED_SEQ = Path(f"../../data/{GENERATED_CLS}/generated/Q/{GENERATED_SID}.fasta")

Homology Modelling: BLDB/FM_0. Min Homology = 0.35. Maximum Number of Templates to Use = 5.


Identy and select template structures for subsequent homology modelling of side chains.

- Use FoldSeek for sequence + structure alignment.
- The homology should be at least 70%.

You may wish to increase the limit on the data rate that the notebook server can send to clients and the window size used to compute the average data rate for the limit, by adding the following tags: 

```
notebook --ServerApp.iopub_data_rate_limit 10000000.0 --ServerApp.rate_limit_window=25.0
```

In [2]:
from time import sleep

from requests import get, post

TICKET_URL = "https://search.foldseek.com/api/ticket"
RESULT_URL = "https://search.foldseek.com/api/result"

# submit a new job
# curl -X POST -F q=@{GEN_PDB} -F 'mode=tmalign' -F 'database[]=pdb100' https://search.foldseek.com/api/ticket
with open(GENERATED_PDB, 'rb') as f:
    ticket = post(
        TICKET_URL,
        files={'q': f},
        data={
            "mode": "tmalign",
            "database[]": "pdb100"
        }).json()
    print("Job submitted:", ticket)

repeat = True
while repeat:
    status = get(f"{TICKET_URL}/{ticket['id']}").json()
    print("Running status:", ticket)
    if status["status"] == "ERROR":
        break
    sleep(1)
    repeat = status["status"] != "COMPLETE"

result = get(f"{RESULT_URL}/{ticket['id']}/0").json()
# import json
# with open("result.json", "w") as json_file:
#     json.dump(result, json_file, indent=4)
result = result["results"][0]["alignments"][0]

visited = set()
templates = [
                {"PDB": pdb,
                 "seq": alignment["tSeq"],
                 "tm-score": alignment["eval"],
                 "prob": alignment["prob"],
                 "seq-per-identity": alignment["seqId"],
                 }
                for alignment in result
                if alignment["seqId"] > MIN_HOMOLOGY
                if alignment["eval"] > MIN_HOMOLOGY
                if (pdb := alignment["target"].split("-")[0].upper()) not in visited and not visited.add(pdb)
                if pdb not in ["6WJM", "6WGP", "2QU1"]
            ][:min(MAX_TEMPLATES, len(result))]

Job submitted: {'id': '3yvS7Hr3b7TyJdzKPW6qfOVSihFBcu9f61DrRA', 'status': 'COMPLETE'}
Running status: {'id': '3yvS7Hr3b7TyJdzKPW6qfOVSihFBcu9f61DrRA', 'status': 'COMPLETE'}


Prepare (1) working directory, (2) FASTAs for alignment, (3) PDBs for homology modelling.

In [3]:
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import PDB, SeqIO
import shutil

wd = Path(f"wd/{GENERATED_CLS}/{GENERATED_SID}")
wd.mkdir(parents=True, exist_ok=True)

pdb_dir = wd / "pdb"
pdb_dir.mkdir(parents=True, exist_ok=True)

fasta = wd / "seqs.fasta"
fasta_seqs = [SeqIO.read(GENERATED_SEQ, "fasta")]
parser = PDB.PDBParser()

shutil.copy(
    src=GENERATED_PDB,
    dst=pdb_dir / GENERATED_PDB.name
)

for temp in templates:
    temp_id = temp["PDB"]
    for temp_pdb in Path(f"../../data/{GENERATED_CLS}/processed").glob(f"{temp_id}*.pdb"):
        print(f"Template {temp_pdb.stem}")
        shutil.copy(
            src=temp_pdb,
            dst=pdb_dir / temp_pdb.name
        )
        structure = parser.get_structure('structure', temp_pdb)
        sequence = ''
        for model in structure:
            for chain in model:
                for residue in chain:
                    if PDB.is_aa(residue, standard=True):
                        sequence += PDB.Polypeptide.index_to_one(PDB.Polypeptide.three_to_index(residue.resname))
        fasta_seqs.append(SeqRecord(Seq(sequence), id=temp_pdb.stem, description=""))

with open(fasta, "w") as f:
    SeqIO.write(fasta_seqs, f, format="fasta")

Template 8DPQ_A
Template 8DON_A
Template 6D7H_A
Template 6W34_B
Template 6W34_A
Template 5TWD_A


Align sequences using Clustal.

In [4]:
# !clustalw -infile={fasta} -output=pir

!clustalo -i {fasta} -o {fasta.parent}/{fasta.stem}.aln.fasta --outfmt=fa
!seqret -sequence {fasta.parent}/{fasta.stem}.aln.fasta -outseq {fasta.parent}/{fasta.stem}.pir -osformat2 pir

Fill the missing details in the PIR alignment.

In [7]:
PIR_RAW = fasta.parent / f"{fasta.stem}.pir"
PIR_REF = PIR_RAW.parent / f"{PIR_RAW.stem}.ref.pir"

entries = []

# l1: >P1;8DOD
# l2: StructureX:8DOD.pdb:26:A:290:::::
# lx: <SEQUENCE>
with open(PIR_RAW, "r") as i, open(PIR_REF, "w") as o:
    entry_id = None
    l2 = False
    for l in i:
        # l1
        if l.startswith(">P1;"):
            entry_id = l[4:].split(" ")[0].strip()
            l2 = True
            o.write(l)
        # l2
        elif l2:
            l2 = False
            chain_id = None
            try:
                chain_id = next(Path(f"../../data/{GENERATED_CLS}/processed").glob(f"{entry_id}*.pdb")).stem.split("_")[-1]
            except StopIteration:
                chain_id = "A"
            start, end = float('inf'), float('-inf')
            for model in parser.get_structure('structure', pdb_dir / f"{entry_id}.pdb"):
                for chain in model:
                    if chain.id != chain_id:
                        continue
                    for residue in chain:
                        if PDB.is_aa(residue, standard=True):
                            start = min(residue.id[1], start)
                            end = max(residue.id[1], start)
            o.write(f"StructureX:{entry_id}.pdb:{start}:{chain_id}:{end}:::::\n")
            entries.append(entry_id)
        # lx
        else:
            o.write(l)

entries.remove(GENERATED_SID)

Homology modelling of side chains.
Main chains are fixed.
Load model from the existing backbone structure.

Model 10 times; select the best one.

You may need to manually edit the data to avoid potential errors.

In [8]:
%cd {wd}

from modeller import *
from modeller.automodel import *
from modeller.scripts import *

# log.verbose()
env = Environ()


class SidechainModel(AutoModel):
    def select_atoms(self):
        return Selection(self).only_sidechain()


env.io.atom_files_directory = "./pdb"

a = SidechainModel(
    env,
    alnfile=PIR_REF.name,
    knowns=entries,
    sequence=GENERATED_SID,
    inifile=str(f"./pdb/{GENERATED_PDB.name}")
)
a.starting_model = 1
a.ending_model = 10
a.assess_methods = [assess.DOPE]
a.make()

/home/tz365/miniconda3/envs/protgen/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


/home/tz365/WS/ProtMatch/notebooks/HM/BLDB/FM_0

                         MODELLER 10.5, 2024/01/19, r12711

     PROTEIN STRUCTURE MODELLING BY SATISFACTION OF SPATIAL RESTRAINTS


                     Copyright(c) 1989-2024 Andrej Sali
                            All Rights Reserved

                             Written by A. Sali
                               with help from
              B. Webb, M.S. Madhusudhan, M-Y. Shen, G.Q. Dong,
          M.A. Marti-Renom, N. Eswar, F. Alber, M. Topf, B. Oliva,
             A. Fiser, R. Sanchez, B. Yerkovich, A. Badretdinov,
                     F. Melo, J.P. Overington, E. Feyfant
                 University of California, San Francisco, USA
                    Rockefeller University, New York, USA
                      Harvard University, Cambridge, USA
                   Imperial Cancer Research Fund, London, UK
              Birkbeck College, University of London, London, UK


Kind, OS, HostName, Kernel, Processor: 4, Linux 23OCT-4090-70

DOPE is a Z-score.

Positive scores are likely to be poor models, while scores lower than -1 or so are likely to be native-like.

Select the best model with the lowest score.

In [9]:
import pandas as pd
import os

lowest_dope = float('inf')
best_model = None

scores = pd.DataFrame([
    {"Model": str(model),
     "DOPE Score": complete_pdb(env, str(model)).assess_normalized_dope()
     } for model in Path("./").glob(f"{GENERATED_SID}.B*.pdb")
]).sort_values(by="DOPE Score", ascending=True)
print(scores)

os.rename(scores.iloc[0]['Model'], f"{GENERATED_SID}.best.pdb")

readlinef__W> File: FM_0.B99990001.pdb, Line: 6
              Modeller will only read the first 80 characters of this line.

>> Model assessment by DOPE potential
iatmcls_286W> MODEL atom not classified:  VAL:OXT  VAL
preppdf_453W> No fixed restraints selected; there may be some dynamic ones.
preppdf_454W> Restraints file was probably not read; use restraints.append().


>> ENERGY; Differences between the model's features and restraints:
Number of all residues in MODEL                   :      270
Number of all, selected real atoms                :     1945    1945
Number of all, selected pseudo atoms              :        0       0
Number of all static, selected restraints         :        0       0
COVALENT_CYS                                      :        F
NONBONDED_SEL_ATOMS                               :        1
Number of non-bonded pairs (excluding 1-2,1-3,1-4):   380873
Dynamic pairs routine                             : 1, NATM x NATM double loop
Atomic shift for contacts up

Verify the modelled side-chains using tools such as:
- PROCHECK
- Verify3D
- WHATCHECK

Some are available at https://saves.mbi.ucla.edu/.